In [9]:
!pip install -U  openai datasets -qq

### [Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://arxiv.org/pdf/2201.11903)

> Abstract: We explore how generating a chain of thought—a series of **intermediate reasoning**
steps—significantly **improves** the ability of large language models to perform
complex reasoning.

<img src="images/cot.png" >

# Step 1: importing openai and writing our completion function

In [18]:
import os
from openai import AzureOpenAI
from llm_config import llm_config, client

def get_completion(messages, model="gpt-35-turbo"):
    completion = client.chat.completions.create(
        model=model,
        messages=messages
        )

    return completion.choices[0].message.content

# Step 2: MMLU dataset

MEASURING MASSIVE MULTITASK LANGUAGE UNDERSTANDING

<img src="images/mmlu.png" >

In [19]:
from datasets import load_dataset

dataset = load_dataset("cais/mmlu","abstract_algebra")

In [20]:
Q1 = dataset['test'][0]
Q1

{'question': 'Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.',
 'subject': 'abstract_algebra',
 'choices': ['0', '4', '2', '6'],
 'answer': 1}

# Step 3: How COT improves the ability of a large language model 

In [25]:
messages=[
            {"role": "system", "content": """you are LLM capable of solving algebra questions given a question \
                                        answer without providing any explanation
                                        """},
            {"role": "user", "content": f"Question: {Q1['question']}"}
        ]
response = get_completion(messages)


In [26]:
print(response)

The degree of the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q is 8.


In [29]:
messages=[
            {"role": "system", "content": "you are LLM capable of solving algebra questions given the following \
            question answer thinking step by step"},
            {"role": "user", "content": f"Question: {Q1['question']}"}
        ]
response = get_completion(messages)

In [30]:
print(response)

Firstly, note that sqrt(18) = sqrt(2)*sqrt(3), so we can simplify the field extension to Q(sqrt(2), sqrt(3)). 

Now, we want to find the degree of this field extension over Q. This means we want to find the dimension of Q(sqrt(2), sqrt(3)) as a vector space over Q. 

Consider the basis {1, sqrt(2), sqrt(3), sqrt(6)}. Every element in Q(sqrt(2), sqrt(3)) can be written as a linear combination of these basis elements with coefficients in Q. 

To show that this basis is indeed a basis, we need to show that these elements are linearly independent and span Q(sqrt(2), sqrt(3)). 

To show linear independence, suppose we have a linear combination of the basis elements equal to zero: 

a + b*sqrt(2) + c*sqrt(3) + d*sqrt(6) = 0 

where a,b,c,d are rational numbers. Then we can rearrange this equation to get: 

d*sqrt(6) = -a - b*sqrt(2) - c*sqrt(3) 

Squaring both sides: 

6d^2 = a^2 + 2ab*sqrt(2) + 2ac*sqrt(3) + 2bc*sqrt(6) + 2b^2 + 3c^2 

Since {1, sqrt(2), sqrt(3), sqrt(6)} are linearly indep

# Step 4: Error analysis with chain of thought

In [33]:
messages=[
            {"role": "system", "content": "you are LLM capable of solving algebra questions by writing a code,\
              write a python code to solve this problem"},
            {"role": "user", "content": f"Question: {Q1['question']}"}
        ]
response = get_completion(messages)

In [34]:
print(response)

We know that Q(sqrt(2)) is a degree-2 extension of Q, since the minimal polynomial of sqrt(2) over Q is x^2 - 2. Similarly, Q(sqrt(3)) is a degree-2 extension of Q.

Now, we need to find the degree of the extension generated by Q(sqrt(2), sqrt(3), sqrt(18)). Note that sqrt(18) = sqrt(2 * 3^2) = sqrt(2) * 3, so we can write:

Q(sqrt(2), sqrt(3), sqrt(18)) = Q(sqrt(2), sqrt(3), 3)

Next, observe that sqrt(3) is not rational but if we add it to the field generated by Q(sqrt(2)), then we obtain a degree-4 extension. This is because the minimal polynomial of sqrt(2) over Q(sqrt(3)) is x^2 - 2, while the minimal polynomial of sqrt(3) over Q(sqrt(2)) is x^2 - 3. Therefore, the minimal polynomial of sqrt(2) and sqrt(3) over Q is the product of these two polynomials, which is:

(x^2 - 2)(x^2 - 3)

This is a degree-4 polynomial, so Q(sqrt(2), sqrt(3)) is a degree-4 extension of Q.

Now, we need to find the degree of the extension generated by Q(sqrt(2), sqrt(3), 3). We can apply the same logic a

In [35]:
!pip install sumpy -qq

from sympy import *

x = symbols('x')
f = (x - 3)*(x**2 - 2)*(x**2 - 3)

f

(x - 3)*(x**2 - 3)*(x**2 - 2)

In [43]:
messages=[
            {"role": "system", "content": """
                   you are LLM capable of solving algebra questions by writing a code,
                   write a python code to colve this problem.
             
                given a question 
             
                # step 1:
                     extract all the given variables and their value
             
                # step 2:
                     write what variable is requested
             
                # step 3:
                     write a formula that you are going to use to solve the problem
             
                # step 4:
                    write the steps to install all the necessery library
             
                # step 5:
                    write a complete python function that solves a problem
             
                # step 6:
                    Answer :  

             """},
            {"role": "user", "content": f"Question: {Q1['question']}"}
        ]
response = get_completion(messages)

In [44]:
print(response)

Step 1: The given variables and their values are:
- p = (1, 2, 5, 4)(2, 3)

Step 2: The variable requested is the index of <p> in S_5.

Step 3: To find the index of a permutation, we can use the formula:
Index(<p>) = σ(1)*(n-1)! + σ(2)*(n-2)! + ... + σ(n-1)*1! + σ(n)*0!
where σ(i) is the number of elements to the right of i in the permutation. The number of elements to the right of n is always 0, hence the final term σ(n)*0!.

Step 4: We don't need to install any library to solve this problem.

Step 5: Here is the Python function that solves the problem:

```python
def find_index_of_permutation(p):
    n = len(p)
    index = 0
    for i in range(n-1):
        sigma_i = sum(1 for j in range(i+1, n) if p[j] < p[i])
        factorial = 1
        for k in range(n-i, 0, -1):
            factorial *= k
        index += sigma_i * factorial
    return index
```

Step 6: To find the index of <p>, we call the function and pass the permutation as an argument:

```python
p = [(1, 2, 5, 4), (2, 3)]

In [38]:
def find_index_of_permutation(p):
    n = len(p)
    index = 0
    for i in range(n-1):
        sigma_i = sum(1 for j in range(i+1, n) if p[j] < p[i])
        factorial = 1
        for k in range(n-i, 0, -1):
            factorial *= k
        index += sigma_i * factorial
    return index

8

# Exercise

repeat what we did for the random question below

In [42]:
Q1 = dataset['test'][1]
Q1

{'question': 'Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the index of <p> in S_5.',
 'subject': 'abstract_algebra',
 'choices': ['8', '2', '24', '120'],
 'answer': 2}